In [1]:
auto file = TFile::Open("../NanoAOD-DYJetsToLL.root")

(TFile *) @0x7f53f5352458


In [2]:
TTree* tree;
file->GetObject("tree", tree);
tree->SetBranchStatus("*", 0);
tree->SetBranchStatus("nMuon", 1);
tree->SetBranchStatus("Muon_pt", 1);
tree->SetBranchStatus("Muon_eta", 1);

In [3]:
#include <ctime>
#include <sys/time.h>

In [4]:
struct timeval startTime, endTime;

In [5]:
double diff(struct timeval endTime, struct timeval startTime) {
  return (1000L * 1000L * (endTime.tv_sec - startTime.tv_sec) + (endTime.tv_usec - startTime.tv_usec)) / 1000.0 / 1000.0;
}

In [6]:
float pz[552056];

In [7]:
ROOT::RDataFrame d("tree", file);

In [8]:
gettimeofday(&startTime, 0);
int k6 = 0;
d.Foreach([&k6](ROOT::VecOps::RVec<float> Muon_pt, ROOT::VecOps::RVec<float> Muon_eta) {
    for (int i = 0;  i < Muon_pt.size();  i++) {
        pz[k6]= Muon_pt[i] * sinh(Muon_eta[i]);
        k6++;
    }
}, {"Muon_pt", "Muon_eta"});
gettimeofday(&endTime, 0);
diff(endTime, startTime)

(double) 0.66133500


0.6396, 0.5379, 0.528, 0.609, 0.546 sec

In [9]:
pz[0]

(float) -153.616f


In [10]:
void doit2(float* pz, TTreeReader &reader, TTreeReaderArray<float> &pt, TTreeReaderArray<float> &eta) {
    int k = 0;
    while (reader.Next()) {
        for (int i = 0;  i < pt.GetSize();  i++) {
            pz[k] = pt[i] * sinh(eta[i]);
            k++;
        }
    }
}

In [13]:
TTreeReader reader3("tree", file);
TTreeReaderArray<float> pt3(reader3, "Muon_pt");
TTreeReaderArray<float> eta3(reader3, "Muon_eta");

gettimeofday(&startTime, 0);
doit2(pz, reader3, pt3, eta3);
gettimeofday(&endTime, 0);
diff(endTime, startTime)

(double) 0.20040700


0.164 the first time, 0.175 the second, then 0.200 sec

In [7]:
UInt_t nMuon;
float pts[10];
float etas[10];
TBranch* nbranch = tree->GetBranch("nMuon");
TBranch* ptbranch = tree->GetBranch("Muon_pt");
TBranch* etabranch = tree->GetBranch("Muon_eta");

In [8]:
tree->SetBranchAddress("nMuon", &nMuon);
tree->SetBranchAddress("Muon_pt", pts);
tree->SetBranchAddress("Muon_eta", etas);

In [9]:
void doit3(float* pz) {
    int k = 0;
    for (int i = 0;  i < 701716;  i++) {
        nbranch->GetEntry(i);
        ptbranch->GetEntry(i);
        etabranch->GetEntry(i);
        for (int j = 0;  j < nMuon;  j++) {
            pz[k] = pts[j] * sinh(etas[j]);
            k++;
        }
    }
}

In [13]:
gettimeofday(&startTime, 0);
doit3(pz);
gettimeofday(&endTime, 0);
diff(endTime, startTime)

(double) 0.14229700


0.131 sec, 0.142, 0.146, 0.142...

In [14]:
struct Muon {
    float pt;
    float eta;
    float phi;
};

struct Event {
    std::vector<Muon> muons;
};

In [15]:
std::vector<Event> events;

In [17]:
TTreeReader reader5("tree", file);
TTreeReaderArray<float> pt5(reader5, "Muon_pt");
TTreeReaderArray<float> eta5(reader5, "Muon_eta");
TTreeReaderArray<float> phi5(reader5, "Muon_phi");

while (reader5.Next()) {
    std::vector<Muon> muons;
    for (int i = 0;  i < pt5.GetSize();  i++) {
        Muon muon;
        muon.pt = pt5[i];
        muon.eta = eta5[i];
        muon.phi = phi5[i];
        muons.push_back(muon);
    }
    Event event;
    event.muons = muons;
    events.push_back(event);
}

In [18]:
events.size()

(unsigned long) 701717


In [19]:
void doit3(float* pz) {
    int k = 0;
    for (int i = 0;  i < events.size();  i++) {
        for (int j = 0;  j < events[i].muons.size();  j++) {
            pz[k] = events[i].muons[j].pt * sinh(events[i].muons[j].eta);
            k++;
        }
    }
}

In [24]:
gettimeofday(&startTime, 0);
doit3(pz);
gettimeofday(&endTime, 0);
diff(endTime, startTime)

(double) 0.11822100


0.1116 sec, 0.0918, 0.113, 0.1178, 0.118